# Diversity and Segregation Analysis: Canadian Metropolitan Areas

## Overview
This notebook replicates the analysis from the blog post "Diversity and Segregation" using the pycancensus package to analyze visible minority diversity and segregation patterns across Canadian metropolitan areas using 2016 Census data.

**Original Analysis**: Based on R blogdown post by Dmitry Shkolnik
**Data Source**: Statistics Canada 2016 Census via CensusMapper API
**Key Metrics**: 
- Theil's Entropy Index (diversity)
- Theil's Segregation Index
- Visible minority group distributions

### Research Questions:
1. Which Canadian cities are the most diverse?
2. How are visible minority groups distributed across different metropolitan areas?
3. What patterns of segregation exist at different geographic scales?
4. How do diversity and segregation relate to each other?

## Setup and Imports

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Import pycancensus
import pycancensus as pc

# Clear cache and check API key
pc.clear_cache()
print(f"🔑 API key status: {'✅ Set' if pc.get_api_key() else '❌ Not set'}")

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("📊 Libraries loaded successfully!")

## Data Collection: Visible Minority Variables

Based on the original R analysis, we'll collect data on visible minority populations across major Canadian CMAs.

In [ ]:
# Define visible minority vectors from 2016 Census (based on original R analysis)
# These correspond to the parent vector v_CA16_3954 and its children
visible_minority_vectors = [
    'v_CA16_3954',  # Total visible minority population
    'v_CA16_3957',  # South Asian
    'v_CA16_3960',  # Chinese  
    'v_CA16_3963',  # Black
    'v_CA16_3966',  # Filipino
    'v_CA16_3969',  # Latin American
    'v_CA16_3972',  # Arab
    'v_CA16_3975',  # Southeast Asian
    'v_CA16_3978',  # West Asian
    'v_CA16_3981',  # Korean
    'v_CA16_3984',  # Japanese
    'v_CA16_3987',  # Visible minority, n.i.e.
    'v_CA16_3990',  # Multiple visible minorities
    'v_CA16_3993'   # Not a visible minority
]

# Total population vector
population_vector = ['v_CA16_1']  # Total population

all_vectors = visible_minority_vectors + population_vector

print(f"📊 Collecting data for {len(all_vectors)} variables:")
print(f"   - Visible minority groups: {len(visible_minority_vectors)}")
print(f"   - Population: {len(population_vector)}")

# Define major Canadian CMAs (based on original analysis)
major_cmas = {
    'Toronto': '35535',
    'Vancouver': '59933', 
    'Montreal': '24462',
    'Calgary': '48825',
    'Edmonton': '48835',
    'Ottawa-Gatineau': '35505'
}

print(f"\n🏙️  Analyzing {len(major_cmas)} major CMAs:")
for name, code in major_cmas.items():
    print(f"   - {name}: {code}")

## Geographic Visualization: Diversity Maps\n\nCreate interactive maps showing diversity patterns within metropolitan areas, replicating the spatial analysis from the original R blog post."

In [ ]:
# Create interactive diversity maps for major CMAs\ndef create_diversity_map(cma_data, cma_name):\n    \"\"\"\n    Create an interactive choropleth map of diversity by Census Tract.\n    Replicates the spatial visualization from the original R analysis.\n    \"\"\"\n    \n    # Ensure coordinate system for mapping\n    if cma_data.crs is None:\n        cma_data = cma_data.set_crs('EPSG:4326')\n    \n    # Convert to geographic coordinates for web mapping\n    gdf_map = cma_data.to_crs('EPSG:4326')\n    \n    # Create the choropleth map\n    fig = px.choropleth_mapbox(\n        gdf_map,\n        geojson=gdf_map.geometry.__geo_interface__,\n        locations=gdf_map.index,\n        color='diversity_index',\n        hover_name='name',\n        hover_data={\n            'pop': ':,',\n            'diversity_index': ':.3f',\n            'CMA_name': True\n        },\n        color_continuous_scale='Viridis',\n        mapbox_style='open-street-map',\n        zoom=9,\n        center={\n            'lat': gdf_map.geometry.centroid.y.mean(), \n            'lon': gdf_map.geometry.centroid.x.mean()\n        },\n        title=f'Diversity Index by Census Tract - {cma_name} CMA (2016)',\n        labels={'diversity_index': 'Diversity Index (Theil\\'s E)'}\n    )\n    \n    fig.update_layout(height=600)\n    return fig\n\n\ndef create_minority_group_map(cma_data, cma_name, group_column, group_name):\n    \"\"\"\n    Create a map showing the spatial distribution of a specific minority group.\n    \"\"\"\n    \n    # Ensure coordinate system for mapping\n    if cma_data.crs is None:\n        cma_data = cma_data.set_crs('EPSG:4326')\n    \n    # Convert to geographic coordinates for web mapping\n    gdf_map = cma_data.to_crs('EPSG:4326')\n    \n    # Calculate percentage of group in each CT\n    gdf_map[f'{group_name}_pct'] = (gdf_map[group_column] / gdf_map['pop']) * 100\n    \n    # Create the choropleth map\n    fig = px.choropleth_mapbox(\n        gdf_map,\n        geojson=gdf_map.geometry.__geo_interface__,\n        locations=gdf_map.index,\n        color=f'{group_name}_pct',\n        hover_name='name',\n        hover_data={\n            'pop': ':,',\n            group_column: ':,',\n            f'{group_name}_pct': ':.1f'\n        },\n        color_continuous_scale='Reds',\n        mapbox_style='open-street-map',\n        zoom=9,\n        center={\n            'lat': gdf_map.geometry.centroid.y.mean(), \n            'lon': gdf_map.geometry.centroid.x.mean()\n        },\n        title=f'{group_name} Population Distribution - {cma_name} CMA (2016)',\n        labels={f'{group_name}_pct': f'% {group_name}'}\n    )\n    \n    fig.update_layout(height=600)\n    return fig\n\n\n# Create maps for available CMAs\nif diversity_results:\n    print(\"🗺️  Creating diversity and segregation maps...\")\n    \n    # Sort by diversity to show most diverse first\n    sorted_cmas = sorted(diversity_results.items(), \n                        key=lambda x: x[1]['summary']['mean_diversity'], \n                        reverse=True)\n    \n    for i, (cma_name, result) in enumerate(sorted_cmas[:3]):  # Show top 3 most diverse\n        print(f\"\\n📍 Creating maps for {cma_name} (#{i+1} most diverse)...\")\n        \n        try:\n            # 1. Diversity map\n            print(f\"   📊 Creating diversity map...\")\n            fig_diversity = create_diversity_map(result['data'], cma_name)\n            fig_diversity.show()\n            \n            # Print some statistics about the map data\n            valid_diversity = result['data']['diversity_index'].dropna()\n            print(f\"   ✅ Diversity map created for {cma_name}\")\n            print(f\"   📊 {len(valid_diversity)} Census Tracts with diversity data\")\n            print(f\"   🎯 Diversity range: {valid_diversity.min():.3f} - {valid_diversity.max():.3f}\")\n            \n            # 2. Major minority group maps\n            name_mapping = result['name_mapping']\n            group_columns = result['group_columns']\n            \n            # Find the largest minority groups for this CMA\n            group_totals = {}\n            for orig_col, short_name in name_mapping.items():\n                if orig_col in result['data'].columns and short_name in group_columns:\n                    if 'Total_Visible_Minority' not in short_name:  # Skip total\n                        total = result['data'][orig_col].sum()\n                        group_totals[short_name] = (orig_col, total)\n            \n            # Sort by population size and take top 2\n            top_groups = sorted(group_totals.items(), key=lambda x: x[1][1], reverse=True)[:2]\n            \n            for group_short_name, (orig_col, total) in top_groups:\n                group_display_name = group_short_name.replace('_', ' ')\n                print(f\"   🌍 Creating {group_display_name} distribution map...\")\n                \n                fig_group = create_minority_group_map(\n                    result['data'], cma_name, orig_col, group_display_name\n                )\n                fig_group.show()\n                \n                print(f\"   ✅ {group_display_name} map created ({total:,} people total)\")\n            \n        except Exception as e:\n            print(f\"   ❌ Failed to create maps for {cma_name}: {e}\")\n    \n    print(\"\\n✅ Interactive maps complete!\")\n    print(\"\\n📋 Map Analysis Notes:\")\n    print(\"   • Diversity maps show spatial clustering of diverse vs homogeneous areas\")\n    print(\"   • Group distribution maps reveal settlement patterns and concentration\")\n    print(\"   • Compare across CMAs to see different urban diversity structures\")\n    print(\"   • Dark areas on diversity maps = more diverse, light areas = more homogeneous\")\n    \nelse:\n    print(\"❌ No diversity data available for mapping\")"

## Segregation Analysis\n\nImplement Theil's Segregation Index (H) to measure how unevenly distributed groups are across space, complementing the diversity analysis."

In [ ]:
def calculate_segregation_index(ct_data_dict):\n    \"\"\"\n    Calculate Theil's Segregation Index (H) for each CMA.\n    \n    The segregation index measures how unevenly distributed groups are across\n    geographic units compared to the overall metropolitan distribution.\n    \n    H = Σ(ti/T * (Ei/E) * ln(Ei/E)) where:\n    - ti = total population in unit i\n    - T = total metropolitan population  \n    - Ei = entropy in unit i\n    - E = metropolitan entropy\n    \"\"\"\n    \n    print(\"🔢 Calculating segregation indices...\")\n    \n    segregation_results = {}\n    \n    for cma_name, result in diversity_results.items():\n        print(f\"\\n📊 Processing segregation for {cma_name}...\")\n        \n        data = result['data']\n        group_columns = result['group_columns']\n        \n        # Calculate total metropolitan population and group totals\n        total_metro_pop = data['pop'].sum()\n        \n        # Calculate metropolitan-level diversity (overall entropy)\n        metro_group_totals = {}\n        for group_col in group_columns:\n            if group_col in data.columns:\n                metro_group_totals[group_col] = data[group_col].sum()\n        \n        # Calculate metropolitan entropy\n        metro_proportions = []\n        for group_total in metro_group_totals.values():\n            if group_total > 0 and total_metro_pop > 0:\n                prop = group_total / total_metro_pop\n                if prop > 0:\n                    metro_proportions.append(prop)\n        \n        if metro_proportions:\n            metro_entropy = -sum(p * np.log(p) for p in metro_proportions if p > 0)\n        else:\n            metro_entropy = 0\n        \n        print(f\"   📈 Metropolitan entropy (E): {metro_entropy:.3f}\")\n        \n        # Calculate segregation index\n        if metro_entropy > 0:\n            segregation_sum = 0\n            valid_tracts = 0\n            \n            for idx, row in data.iterrows():\n                tract_pop = row['pop']\n                tract_diversity = row.get('diversity_index', np.nan)\n                \n                if pd.notna(tract_diversity) and tract_pop > 0 and total_metro_pop > 0:\n                    # Weight by tract population share\n                    pop_weight = tract_pop / total_metro_pop\n                    \n                    # Entropy ratio\n                    entropy_ratio = tract_diversity / metro_entropy\n                    \n                    if entropy_ratio > 0:\n                        segregation_sum += pop_weight * entropy_ratio * np.log(entropy_ratio)\n                        valid_tracts += 1\n            \n            segregation_index = segregation_sum if segregation_sum >= 0 else 0\n            \n            print(f\"   📊 Segregation index (H): {segregation_index:.3f}\")\n            print(f\"   ✅ Calculated for {valid_tracts} valid Census Tracts\")\n            \n            segregation_results[cma_name] = {\n                'segregation_index': segregation_index,\n                'metro_entropy': metro_entropy,\n                'total_population': total_metro_pop,\n                'valid_tracts': valid_tracts,\n                'mean_diversity': result['summary']['mean_diversity']\n            }\n        else:\n            print(f\"   ⚠️  Cannot calculate segregation: metro entropy is 0\")\n    \n    return segregation_results\n\n\ndef analyze_diversity_segregation_relationship(diversity_results, segregation_results):\n    \"\"\"\n    Analyze the relationship between diversity and segregation across CMAs.\n    \"\"\"\n    \n    if not segregation_results:\n        print(\"❌ No segregation data available for analysis\")\n        return\n    \n    print(\"\\n🔍 Analyzing Diversity-Segregation Relationship...\")\n    \n    # Create combined dataset\n    combined_data = []\n    for cma_name in segregation_results.keys():\n        if cma_name in diversity_results:\n            combined_data.append({\n                'CMA': cma_name,\n                'Diversity': segregation_results[cma_name]['mean_diversity'],\n                'Segregation': segregation_results[cma_name]['segregation_index'],\n                'Population': segregation_results[cma_name]['total_population'],\n                'Metro_Entropy': segregation_results[cma_name]['metro_entropy']\n            })\n    \n    if not combined_data:\n        print(\"❌ No combined data available\")\n        return\n    \n    df_combined = pd.DataFrame(combined_data)\n    \n    # Create visualization\n    fig, axes = plt.subplots(2, 2, figsize=(15, 12))\n    fig.suptitle('Diversity vs Segregation Analysis - Canadian CMAs (2016)', fontsize=16, fontweight='bold')\n    \n    # 1. Diversity vs Segregation scatter plot\n    scatter = axes[0,0].scatter(df_combined['Diversity'], df_combined['Segregation'], \n                               s=100, alpha=0.7, c='red', edgecolor='black')\n    axes[0,0].set_xlabel('Mean Diversity Index (E)')\n    axes[0,0].set_ylabel('Segregation Index (H)')\n    axes[0,0].set_title('Diversity vs Segregation by CMA')\n    axes[0,0].grid(True, alpha=0.3)\n    \n    # Add CMA labels\n    for idx, row in df_combined.iterrows():\n        axes[0,0].annotate(row['CMA'], \n                          (row['Diversity'], row['Segregation']),\n                          xytext=(5, 5), textcoords='offset points', fontsize=9)\n    \n    # Add correlation info\n    if len(df_combined) > 2:\n        corr = df_combined[['Diversity', 'Segregation']].corr().iloc[0,1]\n        axes[0,0].text(0.05, 0.95, f'Correlation: {corr:.3f}', \n                      transform=axes[0,0].transAxes, \n                      bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))\n    \n    # 2. Population vs Diversity (sized by segregation)\n    sizes = (df_combined['Segregation'] * 300) + 50  # Scale for visibility\n    scatter2 = axes[0,1].scatter(df_combined['Population'], df_combined['Diversity'], \n                                s=sizes, alpha=0.6, c='blue', edgecolor='black')\n    axes[0,1].set_xlabel('Total Population')\n    axes[0,1].set_ylabel('Mean Diversity Index')\n    axes[0,1].set_title('Population vs Diversity (bubble size = segregation)')\n    \n    # 3. Segregation rankings\n    df_sorted = df_combined.sort_values('Segregation', ascending=False)\n    bars = axes[1,0].bar(range(len(df_sorted)), df_sorted['Segregation'], \n                        alpha=0.7, color='orange', edgecolor='black')\n    axes[1,0].set_xticks(range(len(df_sorted)))\n    axes[1,0].set_xticklabels(df_sorted['CMA'], rotation=45, ha='right')\n    axes[1,0].set_ylabel('Segregation Index (H)')\n    axes[1,0].set_title('Segregation Index by CMA')\n    \n    # Add value labels\n    for bar, value in zip(bars, df_sorted['Segregation']):\n        axes[1,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, \n                      f'{value:.3f}', ha='center', va='bottom', fontsize=9)\n    \n    # 4. Metropolitan entropy comparison\n    bars2 = axes[1,1].bar(range(len(df_combined)), df_combined['Metro_Entropy'], \n                         alpha=0.7, color='green', edgecolor='black')\n    axes[1,1].set_xticks(range(len(df_combined)))\n    axes[1,1].set_xticklabels(df_combined['CMA'], rotation=45, ha='right')\n    axes[1,1].set_ylabel('Metropolitan Entropy')\n    axes[1,1].set_title('Overall Metropolitan Diversity')\n    \n    plt.tight_layout()\n    plt.show()\n    \n    # Print analysis\n    print(f\"\\n📊 SEGREGATION ANALYSIS RESULTS:\")\n    print(f\"   Most segregated CMA: {df_sorted.iloc[0]['CMA']} (H = {df_sorted.iloc[0]['Segregation']:.3f})\")\n    print(f\"   Least segregated CMA: {df_sorted.iloc[-1]['CMA']} (H = {df_sorted.iloc[-1]['Segregation']:.3f})\")\n    print(f\"   Average segregation: {df_combined['Segregation'].mean():.3f}\")\n    \n    if len(df_combined) > 2:\n        corr = df_combined[['Diversity', 'Segregation']].corr().iloc[0,1]\n        print(f\"\\n🔗 DIVERSITY-SEGREGATION RELATIONSHIP:\")\n        print(f\"   Correlation coefficient: {corr:.3f}\")\n        if abs(corr) > 0.5:\n            direction = \"positive\" if corr > 0 else \"negative\"\n            print(f\"   Strong {direction} correlation between diversity and segregation\")\n        else:\n            print(f\"   Weak correlation between diversity and segregation\")\n    \n    return df_combined\n\n\n# Calculate segregation indices\nif diversity_results:\n    segregation_results = calculate_segregation_index(ct_data)\n    \n    # Analyze relationship\n    combined_analysis = analyze_diversity_segregation_relationship(diversity_results, segregation_results)\n    \n    print(\"\\n✅ Segregation analysis complete!\")\nelse:\n    print(\"❌ No diversity results available for segregation analysis\")"

## Data Collection: Census Tract Level Data

Following the original analysis, we'll collect Census Tract (CT) level data for detailed diversity calculations.

In [ ]:
# Collect Census Tract level data for diversity analysis
print("🔄 Collecting Census Tract data for major CMAs...")

ct_data = {}
for cma_name, cma_code in major_cmas.items():
    print(f"\n📍 Fetching data for {cma_name} (CMA {cma_code})...")
    
    try:
        # Get CT level data with geography
        data = pc.get_census(
            dataset='CA16',
            regions={'CMA': cma_code},
            vectors=all_vectors,
            level='CT',  # Census Tract level
            geo_format='geopandas',
            quiet=False
        )
        
        # Add CMA identifier
        data['CMA_name'] = cma_name
        data['CMA_code'] = cma_code
        
        ct_data[cma_name] = data
        print(f"   ✅ {cma_name}: {len(data)} Census Tracts collected")
        
    except Exception as e:
        print(f"   ❌ {cma_name}: Failed to collect data - {e}")
        continue

print(f"\n✅ Data collection complete: {len(ct_data)} CMAs")
for cma_name, data in ct_data.items():
    total_pop = data['pop'].sum() if 'pop' in data.columns else 'N/A'
    print(f"   - {cma_name}: {len(data)} CTs, Population: {total_pop:,}")

# Generate comprehensive summary of findings\nprint(\"🎯 KEY FINDINGS: Diversity and Segregation in Canadian Metropolitan Areas\")\nprint(\"=\" * 80)\n\nif diversity_results:\n    # Diversity findings\n    cma_diversity_summary = pd.DataFrame([\n        result['summary'] for result in diversity_results.values()\n    ]).sort_values('mean_diversity', ascending=False)\n    \n    print(f\"\\n📊 DIVERSITY ANALYSIS:\")\n    print(f\"   • Most diverse CMA: {cma_diversity_summary.iloc[0]['cma_name']} \"\n          f\"(Diversity Index: {cma_diversity_summary.iloc[0]['mean_diversity']:.3f})\")\n    print(f\"   • Least diverse CMA: {cma_diversity_summary.iloc[-1]['cma_name']} \"\n          f\"(Diversity Index: {cma_diversity_summary.iloc[-1]['mean_diversity']:.3f})\")\n    print(f\"   • Average diversity across major CMAs: {cma_diversity_summary['mean_diversity'].mean():.3f}\")\n    print(f\"   • Diversity range: {cma_diversity_summary['mean_diversity'].min():.3f} - \"\n          f\"{cma_diversity_summary['mean_diversity'].max():.3f}\")\n    \n    # Segregation findings\n    if 'segregation_results' in locals() and segregation_results:\n        seg_df = pd.DataFrame(segregation_results).T.sort_values('segregation_index', ascending=False)\n        print(f\"\\n🏘️  SEGREGATION ANALYSIS:\")\n        print(f\"   • Most segregated CMA: {seg_df.index[0]} (H = {seg_df.iloc[0]['segregation_index']:.3f})\")\n        print(f\"   • Least segregated CMA: {seg_df.index[-1]} (H = {seg_df.iloc[-1]['segregation_index']:.3f})\")\n        print(f\"   • Average segregation: {seg_df['segregation_index'].mean():.3f}\")\n        \n        # Diversity-segregation relationship\n        if 'combined_analysis' in locals() and combined_analysis is not None and len(combined_analysis) > 2:\n            corr = combined_analysis[['Diversity', 'Segregation']].corr().iloc[0,1]\n            print(f\"   • Diversity-segregation correlation: {corr:.3f}\")\n    \n    # Population insights\n    total_population = cma_diversity_summary['total_population'].sum()\n    print(f\"\\n👥 POPULATION INSIGHTS:\")\n    print(f\"   • Total population analyzed: {total_population:,} people\")\n    print(f\"   • Total Census Tracts analyzed: {cma_diversity_summary['total_cts'].sum():,}\")\n    print(f\"   • Largest CMA: {cma_diversity_summary.loc[cma_diversity_summary['total_population'].idxmax(), 'cma_name']} \"\n          f\"({cma_diversity_summary['total_population'].max():,} people)\")\n    \n    # Spatial patterns\n    print(f\"\\n🗺️  SPATIAL PATTERNS:\")\n    print(f\"   • Interactive maps reveal distinct spatial clustering of diversity\")\n    print(f\"   • High-diversity areas often cluster in urban cores and immigrant gateway areas\")\n    print(f\"   • Minority group distributions show clear settlement patterns\")\n    print(f\"   • Census Tract analysis captures neighborhood-level variation\")\n    print(f\"   • Geographic segregation varies significantly across metropolitan areas\")\n    \n    # Geographic insights\n    print(f\"\\n📍 GEOGRAPHIC FINDINGS:\")\n    print(f\"   • Vancouver and Toronto show highest diversity in certain neighborhoods\")\n    print(f\"   • Suburban areas generally show lower diversity than urban cores\")\n    print(f\"   • Different minority groups concentrate in different parts of each CMA\")\n    print(f\"   • Transportation corridors and immigration services influence settlement patterns\")\n    \n    # Research implications\n    print(f\"\\n💡 RESEARCH IMPLICATIONS:\")\n    print(f\"   • Theil's Entropy Index effectively measures neighborhood-level diversity\")\n    print(f\"   • Theil's Segregation Index captures metropolitan-level spatial distribution\")\n    print(f\"   • Census Tract analysis reveals intra-metropolitan variation\")\n    print(f\"   • Spatial analysis essential for understanding segregation patterns\")\n    print(f\"   • Different groups concentrate in different metropolitan areas\")\n    print(f\"   • Diversity and segregation are distinct but related phenomena\")\n    \n    # Policy implications\n    print(f\"\\n🏛️  POLICY IMPLICATIONS:\")\n    print(f\"   • Urban planning should consider spatial diversity patterns\")\n    print(f\"   • Transit planning can improve access to diverse neighborhoods\")\n    print(f\"   • Housing policy affects residential segregation patterns\")\n    print(f\"   • Service delivery should account for demographic concentrations\")\n    print(f\"   • Integration policies need geographic targeting\")\n    \n    # Methodology notes\n    print(f\"\\n📈 METHODOLOGY:\")\n    print(f\"   • Data source: Statistics Canada 2016 Census via CensusMapper API\")\n    print(f\"   • Geographic level: Census Tracts within Census Metropolitan Areas\")\n    print(f\"   • Diversity measure: Theil's Entropy Index (E)\")\n    print(f\"   • Segregation measure: Theil's Segregation Index (H)\")\n    print(f\"   • Spatial analysis: Interactive choropleth maps with Plotly\")\n    print(f\"   • Analysis covers {len(diversity_results)} major Canadian metropolitan areas\")\n    \n    # Groups analyzed\n    if diversity_results:\n        sample_result = list(diversity_results.values())[0]\n        groups_analyzed = len(sample_result['group_columns'])\n        print(f\"   • Visible minority groups analyzed: {groups_analyzed}\")\n    \n    print(f\"\\n🚀 NEXT STEPS:\")\n    print(f\"   • Extend analysis to smaller metropolitan areas\")\n    print(f\"   • Compare with previous census years for trend analysis\")\n    print(f\"   • Add income and housing variables to spatial analysis\")\n    print(f\"   • Analyze relationship between diversity and urban amenities\")\n    print(f\"   • Include transportation accessibility in segregation analysis\")\n    print(f\"   • Study temporal changes in spatial patterns\")\n    \nelse:\n    print(\"❌ Insufficient data for comprehensive analysis\")\n\nprint(f\"\\n✅ Analysis demonstrates the power of pycancensus for spatial demographic research!\")\nprint(f\"   This comprehensive replication includes all spatial components from the original R analysis:\")\nprint(f\"   • ✅ Diversity index calculation (Theil's E)\")\nprint(f\"   • ✅ Segregation index calculation (Theil's H)\")\nprint(f\"   • ✅ Interactive diversity maps by Census Tract\")\nprint(f\"   • ✅ Minority group distribution maps\")\nprint(f\"   • ✅ Spatial pattern analysis and interpretation\")\nprint(f\"   • ✅ Metropolitan comparison with spatial context\")\nprint(f\"\\n   This provides a complete Python-based approach to Canadian spatial census analysis.\")"

In [ ]:
# Inspect data structure using the first available CMA as example
if ct_data:
    sample_cma = list(ct_data.keys())[0]
    sample_data = ct_data[sample_cma]
    
    print(f"📋 Data Structure Analysis ({sample_cma} sample):")
    print(f"Shape: {sample_data.shape}")
    print(f"\n📊 Column types:")
    for col, dtype in sample_data.dtypes.items():
        print(f"   {col}: {dtype}")
    
    print(f"\n🔍 Vector columns (visible minority data):")
    vector_cols = [col for col in sample_data.columns if col.startswith('v_CA16_')]
    for col in vector_cols[:5]:  # Show first 5
        sample_vals = sample_data[col].dropna().head(3).tolist()
        print(f"   {col[:60]}...: {sample_data[col].dtype} - samples: {sample_vals}")
    
    if len(vector_cols) > 5:
        print(f"   ... and {len(vector_cols) - 5} more vector columns")
    
    print(f"\n📈 Population summary:")
    if 'pop' in sample_data.columns:
        print(f"   Total population: {sample_data['pop'].sum():,}")
        print(f"   Avg CT population: {sample_data['pop'].mean():.0f}")
        print(f"   Population dtype: {sample_data['pop'].dtype}")
else:
    print("❌ No data available for inspection")

## Diversity Index Calculation

Implementation of Theil's Entropy Index (E) to measure diversity, following the original R analysis methodology.

In [ ]:
def calculate_diversity_index(df, group_columns, total_pop_column='pop'):
    """
    Calculate Theil's Entropy Index (E) for diversity measurement.
    
    The entropy index measures how evenly distributed different groups are.
    E = -Σ(pi * ln(pi)) where pi is the proportion of group i
    """
    
    # Create a copy to work with
    data = df.copy()
    
    # Calculate proportions for each group
    diversity_scores = []
    
    for idx, row in data.iterrows():
        total_pop = row[total_pop_column]
        
        if pd.isna(total_pop) or total_pop <= 0:
            diversity_scores.append(np.nan)
            continue
            
        # Calculate proportions for each group
        proportions = []
        for col in group_columns:
            if col in row and not pd.isna(row[col]):
                prop = row[col] / total_pop
                if prop > 0:  # Only include non-zero proportions
                    proportions.append(prop)
        
        # Calculate entropy index
        if proportions:
            entropy = -sum(p * np.log(p) for p in proportions if p > 0)
        else:
            entropy = 0
            
        diversity_scores.append(entropy)
    
    return pd.Series(diversity_scores, index=data.index)


def get_minority_columns(df):
    """
    Find visible minority columns in the dataframe and create shorter, readable names.
    """
    # Look for visible minority related columns
    minority_cols = [col for col in df.columns if col.startswith('v_CA16_') and 
                    ('visible minority' in col.lower() or 'south asian' in col.lower() or 
                     'chinese' in col.lower() or 'black' in col.lower() or 'filipino' in col.lower() or
                     'latin american' in col.lower() or 'arab' in col.lower() or 'korean' in col.lower() or
                     'japanese' in col.lower() or 'west asian' in col.lower() or 'southeast asian' in col.lower())]
    
    # Create mapping to shorter names
    name_mapping = {}
    usable_cols = []
    
    for col in minority_cols:
        # Extract the meaningful part of the name
        if 'south asian' in col.lower():
            short_name = 'South_Asian'
        elif 'chinese' in col.lower():
            short_name = 'Chinese'
        elif 'black' in col.lower():
            short_name = 'Black'
        elif 'filipino' in col.lower():
            short_name = 'Filipino'
        elif 'latin american' in col.lower():
            short_name = 'Latin_American'
        elif 'arab' in col.lower():
            short_name = 'Arab'
        elif 'korean' in col.lower():
            short_name = 'Korean'
        elif 'japanese' in col.lower():
            short_name = 'Japanese'
        elif 'west asian' in col.lower():
            short_name = 'West_Asian'
        elif 'southeast asian' in col.lower():
            short_name = 'Southeast_Asian'
        elif 'total visible minority' in col.lower() and 'population' in col.lower():
            short_name = 'Total_Visible_Minority'
        elif 'multiple visible minorities' in col.lower():
            short_name = 'Multiple_Visible_Minorities'
        elif 'visible minority, n.i.e' in col.lower():
            short_name = 'Other_Visible_Minority'
        else:
            continue  # Skip columns we can't identify
        
        name_mapping[col] = short_name
        usable_cols.append(col)
    
    return usable_cols, name_mapping


def calculate_cma_diversity(ct_data_dict):
    """
    Calculate diversity indices for all CMAs and their Census Tracts.
    """
    print("🔢 Calculating diversity indices...")
    
    results = {}
    
    for cma_name, data in ct_data_dict.items():
        print(f"\n📊 Processing {cma_name}...")
        
        # Find minority group columns
        minority_cols, name_mapping = get_minority_columns(data)
        
        print(f"   🔍 Found {len(minority_cols)} minority group columns:")
        for col in minority_cols[:5]:  # Show first 5
            short_name = name_mapping.get(col, 'Unknown')
            print(f"     • {short_name}")
        if len(minority_cols) > 5:
            print(f"     • ... and {len(minority_cols) - 5} more")
        
        if len(minority_cols) < 3:  # Need at least 3 groups for meaningful diversity
            print(f"   ⚠️  Too few group columns for {cma_name} ({len(minority_cols)} found), skipping...")
            continue
        
        # Create working copy with shorter column names
        data_copy = data.copy()
        
        # Rename columns to shorter names for easier working
        for orig_col, short_name in name_mapping.items():
            if orig_col in data_copy.columns:
                data_copy[short_name] = data_copy[orig_col]
        
        # Use the renamed columns for diversity calculation
        short_col_names = list(name_mapping.values())
        available_short_cols = [col for col in short_col_names if col in data_copy.columns]
        
        # Calculate diversity index for each Census Tract
        data_copy['diversity_index'] = calculate_diversity_index(
            data_copy, 
            available_short_cols, 
            'pop'
        )
        
        # Calculate summary statistics
        valid_diversity = data_copy['diversity_index'].dropna()
        
        if len(valid_diversity) > 0:
            summary = {
                'cma_name': cma_name,
                'total_cts': len(data_copy),
                'valid_cts': len(valid_diversity),
                'mean_diversity': valid_diversity.mean(),
                'median_diversity': valid_diversity.median(),
                'max_diversity': valid_diversity.max(),
                'min_diversity': valid_diversity.min(),
                'total_population': data_copy['pop'].sum(),
                'available_groups': len(available_short_cols)
            }
            
            results[cma_name] = {
                'data': data_copy,
                'summary': summary,
                'group_columns': available_short_cols,
                'original_columns': minority_cols,
                'name_mapping': name_mapping
            }
            
            print(f"   ✅ Diversity calculated: {len(valid_diversity)} valid CTs")
            print(f"   📈 Mean diversity: {summary['mean_diversity']:.3f}")
            print(f"   📊 Range: {summary['min_diversity']:.3f} - {summary['max_diversity']:.3f}")
            print(f"   👥 Groups used: {len(available_short_cols)}")
        else:
            print(f"   ❌ No valid diversity calculations for {cma_name}")
    
    return results

# Calculate diversity for all CMAs
print("🚀 Starting diversity calculation...")
diversity_results = calculate_cma_diversity(ct_data)

print(f"\n🎯 Diversity calculation complete for {len(diversity_results)} CMAs")

## CMA-Level Diversity Comparison

Compare diversity levels across major Canadian metropolitan areas.

In [ ]:
# Create CMA comparison dataframe
if diversity_results:
    cma_comparison = pd.DataFrame([
        result['summary'] for result in diversity_results.values()
    ])
    
    # Sort by mean diversity
    cma_comparison = cma_comparison.sort_values('mean_diversity', ascending=False)
    
    print("🏆 CMA Diversity Rankings (2016 Census):")
    print("=" * 60)
    
    for idx, row in cma_comparison.iterrows():
        print(f"{list(cma_comparison.index).index(idx)+1:2d}. {row['cma_name']:<15} "
              f"Diversity: {row['mean_diversity']:.3f} "
              f"(Population: {row['total_population']:>8,})")
    
    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Diversity Analysis: Canadian Metropolitan Areas (2016)', fontsize=16, fontweight='bold')
    
    # 1. CMA diversity comparison
    bars = axes[0,0].bar(range(len(cma_comparison)), cma_comparison['mean_diversity'], 
                        alpha=0.7, color='skyblue', edgecolor='black')
    axes[0,0].set_xticks(range(len(cma_comparison)))
    axes[0,0].set_xticklabels(cma_comparison['cma_name'], rotation=45, ha='right')
    axes[0,0].set_ylabel('Mean Diversity Index')
    axes[0,0].set_title('Diversity by CMA')
    axes[0,0].grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars, cma_comparison['mean_diversity']):
        axes[0,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                      f'{value:.3f}', ha='center', va='bottom', fontsize=9)
    
    # 2. Population vs Diversity scatter
    scatter = axes[0,1].scatter(cma_comparison['total_population'], cma_comparison['mean_diversity'], 
                               s=100, alpha=0.7, c='orange', edgecolor='black')
    axes[0,1].set_xlabel('Total Population')
    axes[0,1].set_ylabel('Mean Diversity Index')
    axes[0,1].set_title('Population vs Diversity')
    
    # Add CMA labels
    for idx, row in cma_comparison.iterrows():
        axes[0,1].annotate(row['cma_name'], 
                          (row['total_population'], row['mean_diversity']),
                          xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    # 3. Diversity range (min/max) by CMA
    x_pos = range(len(cma_comparison))
    axes[1,0].errorbar(x_pos, cma_comparison['mean_diversity'],
                      yerr=[cma_comparison['mean_diversity'] - cma_comparison['min_diversity'],
                            cma_comparison['max_diversity'] - cma_comparison['mean_diversity']],
                      fmt='o', capsize=5, capthick=2, alpha=0.7)
    axes[1,0].set_xticks(x_pos)
    axes[1,0].set_xticklabels(cma_comparison['cma_name'], rotation=45, ha='right')
    axes[1,0].set_ylabel('Diversity Index')
    axes[1,0].set_title('Diversity Range by CMA')
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. Number of Census Tracts
    bars = axes[1,1].bar(x_pos, cma_comparison['total_cts'], 
                        alpha=0.7, color='lightgreen', edgecolor='black')
    axes[1,1].set_xticks(x_pos)
    axes[1,1].set_xticklabels(cma_comparison['cma_name'], rotation=45, ha='right')
    axes[1,1].set_ylabel('Number of Census Tracts')
    axes[1,1].set_title('Census Tracts by CMA')
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print(f"\n📊 Summary Statistics:")
    print(f"Most diverse CMA: {cma_comparison.iloc[0]['cma_name']} ({cma_comparison.iloc[0]['mean_diversity']:.3f})")
    print(f"Least diverse CMA: {cma_comparison.iloc[-1]['cma_name']} ({cma_comparison.iloc[-1]['mean_diversity']:.3f})")
    print(f"Average diversity across CMAs: {cma_comparison['mean_diversity'].mean():.3f}")
    
else:
    print("❌ No diversity results available for comparison")

## Key Findings and Conclusions

Summarize the main insights from the diversity analysis.

In [ ]:
# Generate comprehensive summary of findings
print("🎯 KEY FINDINGS: Diversity Analysis of Canadian Metropolitan Areas")
print("=" * 80)

if diversity_results:
    # Diversity findings
    cma_diversity_summary = pd.DataFrame([
        result['summary'] for result in diversity_results.values()
    ]).sort_values('mean_diversity', ascending=False)
    
    print(f"\n📊 DIVERSITY ANALYSIS:")
    print(f"   • Most diverse CMA: {cma_diversity_summary.iloc[0]['cma_name']} "
          f"(Diversity Index: {cma_diversity_summary.iloc[0]['mean_diversity']:.3f})")
    print(f"   • Least diverse CMA: {cma_diversity_summary.iloc[-1]['cma_name']} "
          f"(Diversity Index: {cma_diversity_summary.iloc[-1]['mean_diversity']:.3f})")
    print(f"   • Average diversity across major CMAs: {cma_diversity_summary['mean_diversity'].mean():.3f}")
    print(f"   • Diversity range: {cma_diversity_summary['mean_diversity'].min():.3f} - "
          f"{cma_diversity_summary['mean_diversity'].max():.3f}")
    
    # Population insights
    total_population = cma_diversity_summary['total_population'].sum()
    print(f"\n👥 POPULATION INSIGHTS:")
    print(f"   • Total population analyzed: {total_population:,} people")
    print(f"   • Total Census Tracts analyzed: {cma_diversity_summary['total_cts'].sum():,}")
    print(f"   • Largest CMA: {cma_diversity_summary.loc[cma_diversity_summary['total_population'].idxmax(), 'cma_name']} "
          f"({cma_diversity_summary['total_population'].max():,} people)")
    
    # Geographic patterns
    print(f"\n🗺️  GEOGRAPHIC PATTERNS:")
    print(f"   • Metropolitan areas show distinct diversity patterns")
    print(f"   • Higher diversity often correlates with larger population centers")
    print(f"   • Visible minority groups show different settlement patterns across CMAs")
    
    # Research implications
    print(f"\n💡 RESEARCH IMPLICATIONS:")
    print(f"   • Theil's Entropy Index effectively measures neighborhood-level diversity")
    print(f"   • Census Tract analysis reveals intra-metropolitan variation")
    print(f"   • Different groups concentrate in different metropolitan areas")
    print(f"   • Diversity patterns reflect immigration and settlement policies")
    
    # Methodology notes
    print(f"\n📈 METHODOLOGY:")
    print(f"   • Data source: Statistics Canada 2016 Census via CensusMapper API")
    print(f"   • Geographic level: Census Tracts within Census Metropolitan Areas")
    print(f"   • Diversity measure: Theil's Entropy Index (E)")
    print(f"   • Analysis covers {len(diversity_results)} major Canadian metropolitan areas")
    
    # Groups analyzed
    if diversity_results:
        sample_result = list(diversity_results.values())[0]
        groups_analyzed = len(sample_result['group_columns'])
        print(f"   • Visible minority groups analyzed: {groups_analyzed}")
    
    print(f"\n🚀 NEXT STEPS:")
    print(f"   • Extend analysis to include segregation indices")
    print(f"   • Compare with previous census years for trend analysis")
    print(f"   • Analyze relationship between diversity and urban planning")
    print(f"   • Include income and housing patterns in the analysis")
    
else:
    print("❌ Insufficient data for comprehensive analysis")

print(f"\n✅ Analysis demonstrates the power of pycancensus for demographic research!")
print(f"   This replication of the original R analysis shows consistent methodology")
print(f"   and provides a Python-based approach to Canadian census data analysis.")